# Neural Network

## Perceptrons

In [4]:
from linearalgebra import Vector, dot

In [5]:
def step_function(x: float) -> float:
    return 1.0 if x >= 0 else 0.0

In [6]:
def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
    ''' return 1 if the perceptron "fires",  0 if not'''
    calculation = dot(weights, x) + bias
    return step_function(calculation)

In [7]:
and_weights = [2., 2]
and_bias = -3.

In [8]:
perceptron_output(and_weights, and_bias, [1,1])

1.0

## Feed- forward neural networks

In [9]:
import math

def sigmoid(t: float) -> float:
    return 1/(1+math.exp(-t))

In [10]:
def neuron_output(weights: Vector, inputs: Vector) -> float:
    '''weights includes the bias term, inputs includes a 1'''
    return sigmoid(dot(weights, inputs))

In [11]:
from typing import List

In [17]:
def feed_forward(neural_network: List[List[Vector]],
                 input_vector: Vector) -> List[Vector]:
    '''
    Feeds the inputs vector through the neural network.
    Returns the outputs of all layers (not just the last one).'''
    
    outputs: List[Vector] = []
    
    for layer in neural_network:
        input_with_bias = input_vector + [1]  # Add a constant.
        output = [neuron_output(neuron, input_with_bias)
                  for neuron in layer]
        outputs.append(output)  # add to results.
        #  then the input to the next layers is the output of this one
        
        input_vector = output
        
    return outputs

In [18]:
xor_network = [# hidden layer
                [[20., 20, -30],
                [20., 20, -10]],
                [[-60., 60, -30]]]

In [20]:
feed_forward(xor_network, [1,0])[-1][0]

0.9999999999999059